In [1]:
import os
%pwd
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(self,config_file_path = CONFIG_FILE_PATH,
                      params_file_path = PARAMS_FILE_PATH,
                      schema_file_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config
  
        

In [5]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
           filename,headers = request.urlretrieve(url= self.config.source_url,
                                                   filename=self.config.local_data_file)
           logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, ' r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-01-01 12:58:10,801: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-01 12:58:10,805: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-01 12:58:10,808: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-01 12:58:10,810: INFO: common: created directory at: artifacts]


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/data.zip'